<a href="https://colab.research.google.com/github/Arianr0d/MORD/blob/main/%D0%98%D0%94%D0%972_%D0%9A%D0%90_%D0%9C%D0%9E%D0%A0%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

## Логические операции

In [ ]:
# проверка на длину
def checkErrorLength(a, b):
  if len(a) != len(b):
    return 'Оперция не возможна! Векторы имеют разную длину'

In [ ]:
# операция конъюнкция
def And(a, b):
  checkErrorLength(a, b)
  return [all(i) for i in zip(a, b)]

# операция дизъюнкция
def Or(a ,b):
  checkErrorLength(a, b)
  return [any(i) for i in zip(a, b)]

# операция отрицания
def Not(a):
  return [not i for i in a]

# операция сумма по модулю два
def Xor(a, b):
  checkErrorLength(a, b)
  return [i^j for i, j in zip(a, b)]

# операция эквивалентности
def Equals(a, b):
  checkErrorLength(a, b)
  return [i == j for i, j in zip(a, b)]

# операция импликации
def Implication(a, b):
  checkErrorLength(a, b)
  return Not(And(a, Not(b)))

# операция стрелка Пирса
def PiercesArrow(a, b):
  checkErrorLength(a, b)
  return Not(Or(a, b))

# операция штрих Шеффера
def SchaeffersTouch(a, b):
  checkErrorLength(a, b)
  return Not(And(a, b))

## Формирование вывода

In [ ]:
# функция формирования строки из вектора
def convertColumnsToArrayString(matrix):
  return np.array([''.join(row.astype(int).astype(str)) for row in matrix])

In [ ]:
# формирование результирующей таблицы истинности таблицы переходов
def prettyTableTruthTransition(inputs, states, tableStates):
  tableTransition = pd.DataFrame(data=tableStates.astype(int))
  tableTransition.insert(loc = 0, column = 'v/x', value = convertColumnsToArrayString(np.array(inputs).T))

  columnsName = np.append(
      np.array(['v/x']),
      convertColumnsToArrayString(np.array(states).T)
  )

  tableTransition.columns = columnsName

  return tableTransition

In [ ]:
# формирование результирующей таблицы истинности выходов Мура
def prettyTableTruthOutputForMure(states, outputs):
  tableOutputs = pd.DataFrame(data=outputs.astype(int))
  tableOutputs.insert(loc = 0, column = 'y/x', value = ['Y{}'.format(i+1) for i in range(outputs.shape[0])])

  columnsName = np.append(
      np.array(['y/x']),
      convertColumnsToArrayString(np.array(states).T)
  )

  tableOutputs.columns = columnsName

  return tableOutputs

In [ ]:
# формирование результирующей таблицы истинности выходов Мили
def prettyTableTruthOutputForMili(inputs, states, outputs):
  tableOutputs = pd.DataFrame(data=outputs.astype(int))

  if len(inputs) == 0:
    tableOutputs.insert(loc = 0, column = 'y/x', value = 'Y1')
  else:
    tableOutputs.insert(loc = 0, column = 'v/x', value = convertColumnsToArrayString(np.array(inputs).T))

  columnsName = np.append(
      np.array(['v/x']),
      convertColumnsToArrayString(np.array(states).T)
  )

  tableOutputs.columns = columnsName

  return tableOutputs

## Примеры применения

In [ ]:
v1 = np.array([0, 0, 1, 1]).astype(bool)
v2 = np.array([0, 1, 0, 1]).astype(bool)
x1 = np.array([0, 0, 1, 1]).astype(bool)
x2 = np.array([0, 1, 0, 1]).astype(bool)

sizeState = len(x1)
sizeInput = len(v1)

###Таблицы переходов для КА Мура

In [ ]:
tableStatesForX1Mure = []
tableStatesForX2Mure = []

for index in range(sizeState):
  x1Item = np.array([x1[index]]*sizeInput)
  x2Item = np.array([x2[index]]*sizeInput)

  tableStatesForX1Mure.append(
      Equals(Not(x1Item), Or(x1Item, Xor(x1Item, Or(Or(SchaeffersTouch(v2, v1), x1Item), v1))))
  )

  tableStatesForX2Mure.append(
      Equals(x1Item, And(PiercesArrow(v1, x2Item), Implication(v1, Xor(x2Item, Equals(v1, v2)))))
  )

print('Таблица переходов для состония X1:')
matrixTruthTransitionMure1 = prettyTableTruthTransition([v1, v2], [x1], np.array(tableStatesForX1Mure).T)
matrixTruthTransitionMure1

Таблица переходов для состония X1:


,v/x,0,0,1,1
0,00,1,1,0,0
1,01,1,1,0,0
2,10,1,1,0,0
3,11,1,1,0,0


In [ ]:
print('Таблица переходов для состония X2:')
matrixTruthTransitionMure2 = prettyTableTruthTransition([v1, v2], [x1, x2], np.array(tableStatesForX2Mure).T)
matrixTruthTransitionMure2

Таблица переходов для состония X2:


,v/x,00,01,10,11
0,00,0,1,1,0
1,01,0,1,1,0
2,10,1,1,0,0
3,11,1,1,0,0


### Таблицы выходов для КА Мура

In [ ]:
tableOutputForYMure = np.array([
    Or(x2, Implication(Implication(Not(x2), x1), Xor(Xor(Xor(Xor(x1, x1), x1), x2), x2))),
    And(x2, Equals(PiercesArrow(x1, x2), SchaeffersTouch(x2, SchaeffersTouch(x1, Xor(x2, x2))))),
    PiercesArrow(And(x2, x1), Equals(x1, Implication(Implication(x1, Not(x1)), And(x2, x2)))),
    And(x1, SchaeffersTouch(x2, Equals(Implication(x1, x1), x1))),
])

print('Таблица выходов:')
matrixTruthOutputMure = prettyTableTruthOutputForMure([x1, x2], np.array(tableOutputForYMure).T)
matrixTruthOutputMure

Таблица выходов:


,y/x,00,01,10,11
0,Y1,1,0,0,0
1,Y2,1,1,1,0
2,Y3,1,0,0,1
3,Y4,1,1,0,0


###Таблицы переходов для КА Мили

In [ ]:
tableStatesForX1Mili = []
tableStatesForX2Mili = []

for index in range(sizeState):
  x1Item = np.array([x1[index]]*sizeInput)
  x2Item = np.array([x2[index]]*sizeInput)

  tableStatesForX1Mili.append(
      Xor(x2Item, Xor(v2, And(Not(v1), PiercesArrow(SchaeffersTouch(x2Item, v1), SchaeffersTouch(v1, v2)))))
  )

  tableStatesForX2Mili.append(
      Equals(v2, Implication(x1Item, SchaeffersTouch(x1Item, Xor(And(x2Item, v2), x1Item))))
  )

print('Таблица переходов для состония X1:')
matrixTruthTransitionMili1 = prettyTableTruthTransition([v1, v2], [x2], np.array(tableStatesForX1Mili).T)
matrixTruthTransitionMili1

Таблица переходов для состония X1:


,v/x,0,1,0,1
0,00,0,1,0,1
1,01,1,0,1,0
2,10,0,1,0,1
3,11,1,0,1,0


In [ ]:
print('Таблица переходов для состония X2:')
matrixTruthTransitionMili2 = prettyTableTruthTransition([v2], [x1, x2], np.array(tableStatesForX2Mili).T)
matrixTruthTransitionMili2

Таблица переходов для состония X2:


,v/x,00,01,10,11
0,0,0,0,1,1
1,1,1,1,0,1
2,0,0,0,1,1
3,1,1,1,0,1


### Таблицы выходов для КА Мили

In [ ]:
tableOutputForY1Mili = []
tableOutputForY2Mili = []
tableOutputForY3Mili = []
tableOutputForY4Mili = []

for index in range(sizeState):
  x1Item = np.array([x1[index]]*sizeInput)
  x2Item = np.array([x2[index]]*sizeInput)

  tableOutputForY1Mili.append( Implication(Implication(v1, x1Item), Equals(x1Item, Or(Not(x1Item), v1))) )

  tableOutputForY2Mili.append( Implication(v1, Xor(v1, Equals(v1, Or(Not(x1Item), And(v2, SchaeffersTouch(x2Item, Xor(v1, v2))))))) )

  tableOutputForY4Mili.append( Xor(v2, Equals(x1Item, Implication(x1Item, Implication(x1Item, Or(x1Item, And(SchaeffersTouch(v2, v1), v1)))))) )

tableOutputForY3Mili.append( SchaeffersTouch(Not(x2), Equals(Implication(Implication(And(x1, x1), Not(x1)), x1), x2)) )

print('Таблица выходов для Y1:')
matrixTruthOutputMili1 = prettyTableTruthOutputForMili([v1], [x1], np.array(tableOutputForY1Mili).T)
matrixTruthOutputMili1

Таблица выходов для Y1:


,v/x,0,0,1,1
0,0,0,0,0,0
1,0,0,0,0,0
2,1,1,1,1,1
3,1,1,1,1,1


In [ ]:
print('Таблица выходов для Y2:')
matrixTruthOutputMili2 = prettyTableTruthOutputForMili([v1, v2], [x1, x2], np.array(tableOutputForY2Mili).T)
matrixTruthOutputMili2

Таблица выходов для Y2:


,v/x,00,01,10,11
0,00,1,1,1,1
1,01,1,1,1,1
2,10,0,0,1,1
3,11,0,0,0,0


In [ ]:
print('Таблица выходов для Y3:')
matrixTruthOutputMili3 = prettyTableTruthOutputForMili([], [x1, x2], np.array(tableOutputForY3Mili))
matrixTruthOutputMili3

Таблица выходов для Y3:


,v/x,00,01,10,11
0,Y1,0,1,1,1


In [ ]:
print('Таблица выходов для Y4:')
matrixTruthOutputMili4 = prettyTableTruthOutputForMili([v1, v2], [x1], np.array(tableOutputForY4Mili).T)
matrixTruthOutputMili4

Таблица выходов для Y4:


,v/x,0,0,1,1
0,00,0,0,1,1
1,01,1,1,0,0
2,10,0,0,1,1
3,11,1,1,0,0


##Поиск СКНФ и СДНФ

In [ ]:
# функция построения СКНФ
def printPCNF(inputs, states, tableTransition):
  countRowTransition = tableTransition.shape[0]
  countColumnTransition = tableTransition.shape[1]

  lastKeyInStates = list(states.keys())[len(states) - 1]

  result = 'СКНФ: '

  for rowIndex in range(countRowTransition):
    for colIndex in range(countColumnTransition):
      if not tableTransition[rowIndex, colIndex]:
        result += '('

        for key, input in inputs.items():
          if input[rowIndex]:
            result += 'v{}` ∨ '.format(key)
          else:
            result += 'v{} ∨ '.format(key)

        for key, state in states.items():
          if state[colIndex]:
            result += 'x{}`'.format(key)
          else:
            result += 'x{}'.format(key)

          if lastKeyInStates != key:
            result += ' ∨ '

        result += ')'

  print(result)

In [ ]:
# функция построения СДНФ
def printPDNF(inputs, states, tableTransition):
  countRowTransition = tableTransition.shape[0]
  countColumnTransition = tableTransition.shape[1]

  lastKeyInStates = list(states.keys())[len(states) - 1]

  result = 'СДНФ: '

  for rowIndex in range(countRowTransition):
    for colIndex in range(countColumnTransition):
      if tableTransition[rowIndex, colIndex]:
        result += '('

        for key, input in inputs.items():
          if input[rowIndex]:
            result += 'v{} ∧ '.format(key)
          else:
            result += 'v{}` ∧ '.format(key)

        for key, state in states.items():
          if state[colIndex]:
            result += 'x{}'.format(key)
          else:
            result += 'x{}`'.format(key)

          if lastKeyInStates != key:
            result += ' ∧ '

        result += ')∨'

  if result[len(result) - 1] == '∨':
    result = result[:-1]

  print(result)

## СКНФ и СДНФ для таблицы переходов (по Мили)

In [ ]:
tableTransitionMiliX1 = np.array([[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [1, 1, 1, 1]]).astype(bool)
tableTransitionMiliX2 = np.array([[0, 0, 0, 0], [1, 1, 1, 1], [1, 0, 1, 0], [0, 1, 0, 1]]).astype(bool)

print('x1(t+1)=phi_1(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMiliX1)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMiliX1)
print('\n')

print('x2(t+1)=phi_2(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMiliX2)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMiliX2)

x1(t+1)=phi_1(x1,x2,v1,v2)
СКНФ: (v1 ∨ v2 ∨ x1 ∨ x2)(v1 ∨ v2 ∨ x1 ∨ x2`)(v1 ∨ v2 ∨ x1` ∨ x2)(v1 ∨ v2 ∨ x1` ∨ x2`)(v1 ∨ v2` ∨ x1 ∨ x2)(v1 ∨ v2` ∨ x1 ∨ x2`)(v1 ∨ v2` ∨ x1` ∨ x2)(v1 ∨ v2` ∨ x1` ∨ x2`)(v1` ∨ v2 ∨ x1 ∨ x2)(v1` ∨ v2 ∨ x1 ∨ x2`)(v1` ∨ v2 ∨ x1` ∨ x2)(v1` ∨ v2 ∨ x1` ∨ x2`)
СДНФ: (v1 ∧ v2 ∧ x1` ∧ x2`)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1 ∧ x2)


x2(t+1)=phi_2(x1,x2,v1,v2)
СКНФ: (v1 ∨ v2 ∨ x1 ∨ x2)(v1 ∨ v2 ∨ x1 ∨ x2`)(v1 ∨ v2 ∨ x1` ∨ x2)(v1 ∨ v2 ∨ x1` ∨ x2`)(v1` ∨ v2 ∨ x1 ∨ x2`)(v1` ∨ v2 ∨ x1` ∨ x2`)(v1` ∨ v2` ∨ x1 ∨ x2)(v1` ∨ v2` ∨ x1` ∨ x2)
СДНФ: (v1` ∧ v2 ∧ x1` ∧ x2`)∨(v1` ∧ v2 ∧ x1` ∧ x2)∨(v1` ∧ v2 ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ v2` ∧ x1` ∧ x2`)∨(v1 ∧ v2` ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2)


## СКНФ и СДНФ для таблицы выходов (по Мили)

In [ ]:
tableOutputMiliY1 = np.array([[1, 1, 1, 1], [1, 0, 1, 1], [0, 0, 1, 1], [0, 1, 1, 1]]).astype(bool)
tableOutputMiliY2 = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 0, 1, 1], [0, 1, 1, 1]]).astype(bool)
tableOutputMiliY3 = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]).astype(bool)
tableOutputMiliY4 = np.array([[1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1]]).astype(bool)

print('y1(t)=psi_1(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY1)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY1)
print('\n')

print('y2(t)=psi_2(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY2)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY2)
print('\n')

print('y3(t)=psi_3(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY3)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY3)
print('\n')

print('y4(t)=psi_4(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY4)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableOutputMiliY4)

y1(t)=psi_1(x1,x2,v1,v2)
СКНФ: (v1 ∨ v2` ∨ x1 ∨ x2`)(v1` ∨ v2 ∨ x1 ∨ x2)(v1` ∨ v2 ∨ x1 ∨ x2`)(v1` ∨ v2` ∨ x1 ∨ x2)
СДНФ: (v1` ∧ v2` ∧ x1` ∧ x2`)∨(v1` ∧ v2` ∧ x1` ∧ x2)∨(v1` ∧ v2` ∧ x1 ∧ x2`)∨(v1` ∧ v2` ∧ x1 ∧ x2)∨(v1` ∧ v2 ∧ x1` ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ v2` ∧ x1 ∧ x2`)∨(v1 ∧ v2` ∧ x1 ∧ x2)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1 ∧ x2)


y2(t)=psi_2(x1,x2,v1,v2)
СКНФ: (v1` ∨ v2 ∨ x1 ∨ x2`)(v1` ∨ v2` ∨ x1 ∨ x2)
СДНФ: (v1` ∧ v2` ∧ x1` ∧ x2`)∨(v1` ∧ v2` ∧ x1` ∧ x2)∨(v1` ∧ v2` ∧ x1 ∧ x2`)∨(v1` ∧ v2` ∧ x1 ∧ x2)∨(v1` ∧ v2 ∧ x1` ∧ x2`)∨(v1` ∧ v2 ∧ x1` ∧ x2)∨(v1` ∧ v2 ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ v2` ∧ x1` ∧ x2`)∨(v1 ∧ v2` ∧ x1 ∧ x2`)∨(v1 ∧ v2` ∧ x1 ∧ x2)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1 ∧ x2)


y3(t)=psi_3(x1,x2,v1,v2)
СКНФ: 
СДНФ: (v1` ∧ v2` ∧ x1` ∧ x2`)∨(v1` ∧ v2` ∧ x1` ∧ x2)∨(v1` ∧ v2` ∧ x1 ∧ x2`)∨(v1` ∧ v2` ∧ x1 ∧ x2)∨(v1` ∧ v2 ∧ x1` ∧ x2`)∨(v1` ∧ v2 ∧ x1` ∧ x2)∨(v1` ∧ v2 ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ 

## СКНФ и СДНФ для таблицы переходов (по Муру)

In [ ]:
tableTransitionMureX1 = np.array([[0, 1, 1, 0], [0, 1, 0, 1], [1, 1, 0, 0], [1, 1, 1, 1]]).astype(bool)
tableTransitionMureX2 = np.array([[0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1], [0, 1, 1, 1]]).astype(bool)

print('x1(t+1)=phi_1(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMureX1)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMureX1)
print('\n')

print('x2(t+1)=phi_2(x1,x2,v1,v2)')
printPCNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMureX2)
printPDNF({1: v1, 2: v2}, {1: x1, 2: x2}, tableTransitionMureX2)

x1(t+1)=phi_1(x1,x2,v1,v2)
СКНФ: (v1 ∨ v2 ∨ x1 ∨ x2)(v1 ∨ v2 ∨ x1` ∨ x2`)(v1 ∨ v2` ∨ x1 ∨ x2)(v1 ∨ v2` ∨ x1` ∨ x2)(v1` ∨ v2 ∨ x1` ∨ x2)(v1` ∨ v2 ∨ x1` ∨ x2`)
СДНФ: (v1` ∧ v2` ∧ x1` ∧ x2)∨(v1` ∧ v2` ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1` ∧ x2)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ v2` ∧ x1` ∧ x2`)∨(v1 ∧ v2` ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1` ∧ x2`)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1 ∧ x2)


x2(t+1)=phi_2(x1,x2,v1,v2)
СКНФ: (v1 ∨ v2 ∨ x1 ∨ x2)(v1 ∨ v2` ∨ x1 ∨ x2)(v1` ∨ v2 ∨ x1 ∨ x2)(v1` ∨ v2` ∨ x1 ∨ x2)
СДНФ: (v1` ∧ v2` ∧ x1` ∧ x2)∨(v1` ∧ v2` ∧ x1 ∧ x2`)∨(v1` ∧ v2` ∧ x1 ∧ x2)∨(v1` ∧ v2 ∧ x1` ∧ x2)∨(v1` ∧ v2 ∧ x1 ∧ x2`)∨(v1` ∧ v2 ∧ x1 ∧ x2)∨(v1 ∧ v2` ∧ x1` ∧ x2)∨(v1 ∧ v2` ∧ x1 ∧ x2`)∨(v1 ∧ v2` ∧ x1 ∧ x2)∨(v1 ∧ v2 ∧ x1` ∧ x2)∨(v1 ∧ v2 ∧ x1 ∧ x2`)∨(v1 ∧ v2 ∧ x1 ∧ x2)


## СКНФ и СДНФ для таблицы выходов (по Муру)

In [ ]:
tableOutputMureY1 = np.array([[1, 1, 1, 1]]).astype(bool)
tableOutputMureY2 = np.array([[0, 0, 1, 0]]).astype(bool)
tableOutputMureY3 = np.array([[1, 1, 0, 1]]).astype(bool)

print('y1(t)=psi_1(x1,x2)')
printPCNF({}, {1: x1, 2: x2}, tableOutputMureY1)
printPDNF({}, {1: x1, 2: x2}, tableOutputMureY1)
print('\n')

print('y2(t)=psi_2(x1,x2)')
printPCNF({}, {1: x1, 2: x2}, tableOutputMureY2)
printPDNF({}, {1: x1, 2: x2}, tableOutputMureY2)
print('\n')

print('y3(t)=psi_3(x1,x2)')
printPCNF({}, {1: x1, 2: x2}, tableOutputMureY3)
printPDNF({}, {1: x1, 2: x2}, tableOutputMureY3)

y1(t)=psi_1(x1,x2)
СКНФ: 
СДНФ: (x1` ∧ x2`)∨(x1` ∧ x2)∨(x1 ∧ x2`)∨(x1 ∧ x2)


y2(t)=psi_2(x1,x2)
СКНФ: (x1 ∨ x2)(x1 ∨ x2`)(x1` ∨ x2`)
СДНФ: (x1 ∧ x2`)


y3(t)=psi_3(x1,x2)
СКНФ: (x1` ∨ x2)
СДНФ: (x1` ∧ x2`)∨(x1` ∧ x2)∨(x1 ∧ x2)


### Матрица соединений

In [ ]:
def unionSizeAllMatrixes(matrixes):
  if len(matrixes) == 0:
    print('Матрица пустая!')
    return

  countRow = matrixes[0].shape[0]
  countColumn = matrixes[0].shape[1]

  for matrix in matrixes:
    if matrix.shape[0] != countRow and matrix.shape[1] != countColumn:
      print('Размеры матриц не совпадают')
      return

  return (countRow, countColumn)

In [ ]:
# функция объединения матриц переходов
def unionMatrix(matrixes):
    countMatrixes = len(matrixes)
    countRow, countColumn = unionSizeAllMatrixes(matrixes)
    result = []

    for row in range(countRow):
      rowMatrix = []

      for column in range(countColumn):
        itemInAllMatrix = np.array([matrixes[index][row, column] for index in range(countMatrixes)])
        rowMatrix.append(''.join(itemInAllMatrix.astype(str)))

      result.append(rowMatrix)

    return np.array(result)

In [ ]:
# функция реализации таблицы соединений для Мили
def printMatrixСonnectionMili(inputs, states, matrixTranstion, matrixOutput):
    countRow, countColumn = matrixTranstion.shape
    tableConnection = pd.DataFrame('0', index=states, columns=states)

    for row in range(countRow):
      indexRowInput = inputs[row].astype(str)

      for column in range(countColumn):
        indexStartState = states[column].astype(str)
        nextState = matrixTranstion[row, column]

        if tableConnection[indexStartState][nextState] != '0':
          tableConnection[indexStartState][nextState] += ' ∧ ' + indexRowInput + '/' + matrixOutput[row, column]
        else:
          tableConnection[indexStartState][nextState] = indexRowInput + '/' + matrixOutput[row, column]

    return tableConnection

In [ ]:
# функция реализации таблицы соединений для Мура
def printMatrixСonnectionMura(inputs, states, matrixTranstion):
    countRow, countColumn = matrixTranstion.shape
    tableConnection = pd.DataFrame('0', index=states, columns=states)

    for row in range(countRow):
      indexRowInput = inputs[row].astype(str)

      for column in range(countColumn):
        indexStartState = states[column].astype(str)
        nextState = matrixTranstion[row, column]

        if tableConnection[indexStartState][nextState] != '0':
          tableConnection[indexStartState][nextState] += ' ∧ ' + indexRowInput
        else:
          tableConnection[indexStartState][nextState] = indexRowInput

    return tableConnection

##Применение таблицы соединений

In [ ]:
# матрица соединений для Мили (в задаче по построению таблицы истинности)
printMatrixСonnectionMili(
    convertColumnsToArrayString(np.array([v1, v2]).T),
    convertColumnsToArrayString(np.array([x1, x2]).T),
    unionMatrix(np.array([
        np.array(matrixTruthTransitionMili1)[:, 1:],
        np.array(matrixTruthTransitionMili2)[:, 1:],
    ])),
    unionMatrix(np.array([
        np.array(matrixTruthOutputMili1)[:, 1:],
        np.array(matrixTruthOutputMili2)[:, 1:],
        np.tile(np.array(matrixTruthOutputMili3)[:, 1:],(4,1)),
        np.array(matrixTruthOutputMili4)[:, 1:],
    ])),
)

,00,01,10,11
00,00/0100 ∧ 10/1000,0,0,0
01,0,01/0111 ∧ 11/1011,00/0111 ∧ 10/1111,01/0110 ∧ 11/1010
10,0,00/0110 ∧ 10/1010,01/0110 ∧ 11/1010,0
11,01/0101 ∧ 11/1001,0,0,00/0111 ∧ 10/1111


In [ ]:
# матрица соединений для Мили (в задаче по построению СКНФ и СДНФ)
printMatrixСonnectionMili(
    convertColumnsToArrayString(np.array([v1, v2]).T),
    convertColumnsToArrayString(np.array([x1, x2]).T),
    unionMatrix(np.array([
        tableTransitionMiliX1.astype(int),
        tableTransitionMiliX2.astype(int),
    ])),
    unionMatrix(np.array([
        tableOutputMiliY1.astype(int),
        tableOutputMiliY2.astype(int),
        tableOutputMiliY3.astype(int),
        tableOutputMiliY4.astype(int),
    ])),
)

,00,01,10,11
00,00/1111,00/1111 ∧ 10/0011,00/1111,00/1111 ∧ 10/1111
01,01/1111 ∧ 10/0111,01/0111,01/1111 ∧ 10/1111,01/1111
10,11/0011,0,11/1111,0
11,0,11/1111,0,11/1111


In [ ]:
# матрица соединений для Мура (в задаче по построению таблицы истинности)
printMatrixСonnectionMura(
    convertColumnsToArrayString(np.array([v1, v2]).T),
    convertColumnsToArrayString(np.array([x1, x2]).T),
    unionMatrix(np.array([
        np.array(matrixTruthTransitionMure1)[:, 1:],
        np.array(matrixTruthTransitionMure2)[:, 1:],
    ])),
)

,00,01,10,11
00,0,0,10 ∧ 11,00 ∧ 01 ∧ 10 ∧ 11
01,0,0,00 ∧ 01,0
10,00 ∧ 01,0,0,0
11,10 ∧ 11,00 ∧ 01 ∧ 10 ∧ 11,0,0


In [ ]:
# матрица соединений для Мура (в задаче по построению СКНФ и СДНФ)
printMatrixСonnectionMura(
    convertColumnsToArrayString(np.array([v1, v2]).T),
    convertColumnsToArrayString(np.array([x1, x2]).T),
    unionMatrix(np.array([
        tableTransitionMureX1.astype(int),
        tableTransitionMureX2.astype(int),
    ])),
)

,00,01,10,11
00,00 ∧ 01,0,0,0
01,0,0,01 ∧ 10,00 ∧ 10
10,10 ∧ 11,0,0,0
11,0,00 ∧ 01 ∧ 10 ∧ 11,00 ∧ 11,01 ∧ 11
